<img align="left" src="media/Assets&ArchHeader.jpg">

# Managing the Db2 Data Management Console

This Jupyter Notebook contains examples of how to setup and manage the Db2 Data Management Console. It covers how to add additional users using database authentication, how to explore and manage connections and setup and manage monitoring profiles.

The Db2 Data Management Console is more than a graphical user interface. It is a set of microservices that you can use to script your use of Db2.

<img align="left" src="media/DMC.png">

### Launching the Db2 Console
You can launch the Db2 Console using the browser running in the Virtual desktop or from your own desktop. 

Inside the virtual machine desktop you can follow the link: http://localhost:11080.

To use the Db2 Console from your own browser, look for the link in your welcome note. It will start with **services-uscentral.skytap.com**

You can log into the console using:
* userid: db2inst1
* password: db2inst1

### Db2 Console Open API
This Jupyter Notebook contains examples of how to use the Open APIs and the composable interface that are available in the Db2 Data Management Console. Everything in the User Interface is also available through an open and fully documented RESTful Services API. The full set of APIs are documented as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to the documentation directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

### Where to find this sample online
You can find a copy of this notebook at https://github.com/Db2-DTE-POC/db2dmc.

### First we will import a few helper classes
We need to pull in a few standard Python libraries so that we can work with REST, JSON and a library called Pandas. Pandas lets us work with DataFrames, which are a very powerful way to work with tabular data in Python. 

In [12]:
# Import the class libraries 
import requests
import ssl
import json
from pprint import pprint
from requests import Response
import pandas as pd
import time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from IPython.display import IFrame
from IPython.display import display, HTML
from pandas import json_normalize
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

### The Db2 Class
Next we will create a Db2 helper class that will encapsulate the Rest API calls that we can use to directly access the Db2 Data Management Console service without having to use the user interface. 

To access the service we need to first authenticate with the service and create a reusable token that we can use for each call to the service. This ensures that we don't have to provide a userID and password each time we run a command. The token makes sure this is secure. 

Each request is constructed of several parts. First, the URL and the API identify how to connect to the service. Second the REST service request that identifies the request and the options. For example '/metrics/applications/connections/current/list'. And finally some complex requests also include a JSON payload. For example running SQL includes a JSON object that identifies the script, statement delimiters, the maximum number of rows in the results set as well as what do if a statement fails.

The full set of APIs are documents as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to that directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

In [13]:
# Run the Db2 Class library
# Used to construct and reuse an Autentication Key
# Used to construct RESTAPI URLs and JSON payloads
class Db2Console():
    
    def __init__(self, url, verify = False, proxies=None, ):
        self.url = url
        self.proxies = proxies
        self.verify = verify

    def authenticate(self, userid, password, profile=""):
        credentials = {'userid':userid, 'password':password}
        r = requests.post(self.url+'/auth/tokens', verify=self.verify, json=credentials, proxies=self.proxies)
        if (r.status_code == 200):
            bearerToken = r.json()['token']
            if profile == "":
                self.headers = {'Authorization': 'Bearer'+ ' '+bearerToken}
                return True;
            else:
                self.headers = {'Authorization': 'Bearer'+ ' '+bearerToken, 'X-DB-Profile': profile}
                return True;
        else:
            print ('Unable to authenticate, no bearer token obtained')
            return False;
        
    def printResponse(self, r, code):
        if (r.status_code == code):
            pprint(r.json())
        else:
            print (r.status_code)
            print (r.content)
    
    def getRequest(self, api, json=None):
        return requests.get(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json)

    def postRequest(self, api, json=None):
        return requests.post(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
    
    def deleteRequest(self, api, json=None):
        return requests.delete(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
    
    def putRequest(self, api, json=None):
        return requests.put(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
        
    def getStatusCode(self, response):
        return (response.status_code)

    def getJSON(self, response):
        return (response.json())
    
    def runSQL(self, script, limit=10, separator=';', stopOnError=False):
        sqlJob = {'commands': script, 'limit':limit, 'separator':separator, 'stop_on_error':str(stopOnError)}
        return self.postRequest('/sql_jobs',sqlJob)
        
    def getSQLJobResult(self, jobid):
        return self.getRequest('/sql_jobs/'+jobid)
    
    def getUserPriviledges(self, profile=''):
        if profile == '' :
            return self.getRequest('/userProfilePrivileges')
        else : 
            return self.getRequest('/userProfilePrivileges/'+profile)
    
    def assignUserPrivileges(self, profile, user):
        json = [{'profileName': profile, 'USER':[user], 'OWNER':[]}]
        return self.postRequest('/userProfilePrivileges?action=assign', json) 
 
    def assignOwnerPrivileges(self, profile, owner):
        json = [{'profileName': profile, 'USER':[], 'OWNER':[owner]}]
        return self.postRequest('/userProfilePrivileges?action=assign', json) 
    
    def revokeProfilePrivileges(self, profile, user):
        json = [{'profileName': profile, 'USER':[user]}]
        return self.postRequest('/userProfilePrivileges?action=revoke', json) 

    def getConnectionProfile(self,profile):
        return self.getRequest('/dbprofiles/'+profile)    
    
    def getMonitorStatus(self):
        return self.getRequest('/monitor') 
    
    def getConnectionProfiles(self):
        return self.getRequest('/dbprofiles')  
    
    def getConsoleRepository(self):
        return self.getRequest('/repository')    

    def getExportConnectionProfiles(self):
        return self.getRequest('/dbprofiles/transfer/export?exportCred=true')
    
    def postConnectionProfile(self, connectionName, dbName, port, host, userid, password, comment):
        json = {"name":connectionName,"location":"","databaseName":dbName,"dataServerType":"DB2LUW","port":port,"host":host,"URL":"jdbc:db2://"+host+":"+port+"/"+dbName+":retrieveMessagesFromServerOnGetMessage=true;","sslConnection":"false","disableDataCollection":"false","collectionCred":{"securityMechanism":"3","user":userid,"password":password},"operationCred":{"securityMechanism":"3","user":userid,"password":password,"saveOperationCred":"true"},"comment":comment}
        return self.postRequest('/dbprofiles', json)
    
    def putConnectionProfileUpdate(self, connectionName, dbName, port, host, userid, password, comment):
        json = {"name":connectionName,"location":"","databaseName":dbName,"dataServerType":"DB2LUW","port":port,"host":host,"URL":"jdbc:db2://"+host+":"+port+"/"+dbName+":retrieveMessagesFromServerOnGetMessage=true;","sslConnection":"false","disableDataCollection":"false","collectionCred":{"securityMechanism":"3","user":userid,"password":password},"operationCred":{"securityMechanism":"3","user":userid,"password":password,"saveOperationCred":"true"},"comment":comment}
        return self.putRequest('/dbprofiles/'+connectionName, json)
    
    def postTestConnection(self, dbName, port, host, userid, password):
        json = {"name":"","location":"","databaseName":dbName,"dataServerType":"DB2LUW","port":port,"host":host,"URL":"jdbc:db2://"+host+":"+port+"/"+dbName+":retrieveMessagesFromServerOnGetMessage=true;","sslConnection":"false","disableDataCollection":"false","operationCred":{"securityMechanism":"3","user":userid,"password":password}}
        return self.postRequest('/dbprofiles/testConnection', json)
    
    def deleteConnectionProfile(self, connectionName):
        return self.deleteRequest('/dbprofiles/'+connectionName)

    def getMonitoringProfiles(self):
        return self.getRequest('/monitorprofile/front')
    
    def getMonitoringProfile(self, profileID):
        return self.getRequest('/monitorprofile/front/'+profileID)
    
    def putMonitoringProfile(self, profileID, json):
        return self.putRequest('/monitorprofile/front/'+profileID, json)   
    
    def getProfileIndex(self, profileName):
        r = self.getMonitoringProfiles()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r) 
            profileList = pd.DataFrame(json_normalize(json['resources']))[['name','id']]
            display(profileList)
            profileList.set_index('name',inplace=True)
            try:
                profileIndex = profileList.loc[profileName][0]
            except KeyError:
                profileIndex = 0
                print(profileName + " not found")
            return profileIndex
        else:
            print(self.getStatusCode(r)) 

### Db2 Data Management Console Connection
To connect to the Db2 Data Management Console service you need to provide the URL, the service name (v4) and profile the console user name and password as well as the name of the connection profile used in the console to connect to the database you want to work with. For this lab we are assuming that the following values are used for the connection:
* Userid: db2inst1
* Password: db2inst1
* Connection: sample

**Note:** If the Db2 Data Management Console has not completed initialization, the connection below will fail. Wait for a few moments and then try it again.

In [14]:
# Connect to the Db2 Data Management Console service
Console  = 'http://localhost:11080'
user     = 'DB2INST1'
password = 'db2inst1'

# Set up the required connection
databaseAPI = Db2Console(Console+'/dbapi/v4')
if databaseAPI.authenticate(user, password) :
    print("Token Created")
else : 
    print("Token Creation Failed")
database = Console

Token Created


### Confirm the connection
To confirm that your connection is working you can the Console connection profiles.

In [15]:
# Get Console Connection Profiles
r = databaseAPI.getConnectionProfiles()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)))
else:
    print(databaseAPI.getStatusCode(r))      

,name,disableDataCollection,databaseVersion,databaseName,timeZone,DB_IDENTITY_HASH,databaseVersion_VRMF,sslConnection,userProfileRole,host,xtraProps,_PROFILE_INIT_,dataServerType,port,edition,dataServerExternalType,unsupport,capabilities,otsEventTablespace
0,Repository,false,11.5.0,REPO,-50000,-1276862475,11.5.5.0,false,OWNER,localhost,,true,DB2LUW,50001,ADV,DB2LUW,,"[""DSM_ENTERPRISE_LUW""]",TS4MONITOR
1,Banking-ML,false,11.5.0,BANKING,-50000,83780771,11.5.5.0,false,OWNER,localhost,,true,DB2LUW,50001,ADV,DB2LUW,,"[""DSM_ENTERPRISE_LUW""]",TS4MONITOR
2,Sample,false,11.5.0,SAMPLE,-50000,1320657261,11.5.5.0,false,OWNER,localhost,,true,DB2LUW,50001,ADV,DB2LUW,,"[""DSM_ENTERPRISE_LUW""]",TS4MONITOR
3,Ontime,false,11.5.0,ONTIME,-50000,1218348271,11.5.5.0,false,OWNER,localhost,,true,DB2LUW,50001,ADV,DB2LUW,,"[""DSM_ENTERPRISE_LUW""]",TS4MONITOR


### Get Repository Configuration
You can also get details on the repository configuration. You can see that we are using a local database name HISTORY to store all the monitoring data collected by the console. 

In [16]:
# Get Console Repository Configuration
r = databaseAPI.getConsoleRepository()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r) 
    display(pd.DataFrame(json_normalize(json)[['databaseName','status','host','port','collectionCred.user']]).transpose())
else:
    print(databaseAPI.getStatusCode(r))      

,0
databaseName,REPO
status,active
host,localhost
port,50001
collectionCred.user,db2inst1


### Running SQL Routines
We need to run some SQL scripts later in the lab. So run the next two cells. They define routines that run SQL scripts and display the results of those scripts. If you want to learn more about how these routines were developed, check out the [Analyzing SQL Workloads notebook](http://localhost:8888/notebooks/Db2_Data_Management_Console_SQL.ipynb).

In [17]:
def runSQL(profile,user, password, sqlText):
    
    if databaseAPI.authenticate(user, password, profile) :

        runID = databaseAPI.getJSON(databaseAPI.runSQL(sqlText))['id'] 

        json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
        while 'results' not in json :
                json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
        fulljson = json

        while json['results'] != [] or (json['status'] != "completed" and json['status'] != "failed") :
            json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
            while 'results' not in json :
                json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
            for results in json['results'] :
                fulljson['results'].append(results)
            time.sleep(1) 
        return fulljson
    else :
        print('Could not authenticate')
print('runSQL routine defined')

runSQL routine defined


In [18]:
def displayResults(json):

    for results in json['results']:
        print('Statement: '+str(results['index'])+': '+results['command'])
        if 'error' in results : 
            print(results['error'])
        elif 'rows' in results :
            df = pd.DataFrame(results['rows'],columns=results['columns'])
            print(df)
        else :
            print('No errors. Row Affected: '+str(results['rows_affected']))
        print()
print('displayResults routine defined')

displayResults routine defined


## Accessing the Db2 Console through custom URLs
This lab provides direct links to specific pages in the Db2 Console to make it easy to navigate during the lab. Since you can use this lab from a browser running in the virtual desktop or from your own browser on your own desktop, we just need to save the location of the console you are using. 

By default it is setup to use links that will work from the virtual machine desktop. To run from your own browser on your own desktop, enter the port location of the Db2 Console provided in your welcome note and run the cell below.

In [22]:
externalConsole = "http://services-uscentral.skytap.com:18638"
database = externalConsole

### Testing the Custom URL
Run the cell below. 

It will generate a link to the console that works for your selected environment. Notice that it brings up the whole console including the full navigation menu. 

Right click on the link and select **Open Link in New Tab** or **Open Link in New Window**

In [24]:
print(database+'/console')

http://services-uscentral.skytap.com:18638/console


You can also navigate to a specific page in the console by specifying the full URL or the page. For example, run the following cell and click on the link to navigate to the page that displays the tables in the Sample database:

In [25]:
print(database+'/console/#explore/table?profile=Sample')

http://services-uscentral.skytap.com:18638/console/#explore/table?profile=Sample


You can also choose to create a URL that just links a page without the full navigation menu. Simply add **?mode=compact** to the URL as an option. For example:

In [26]:
print(database+'/console/?mode=compact#explore/table?profile=Sample')

http://services-uscentral.skytap.com:18638/console/?mode=compact#explore/table?profile=Sample


## Setting up Repository Authentication

### Reviewing current console authentication and switching to repository authentication

Let's start by looking at the current authentication settings in the console. To see the authentication settings you can either:

Use the full console to switch to Repository Database Authentication:
1. Open the Console
2. Click the gear icon at the top right of the page
3. Select **Authentication Setting**

Use the embedded page from the console to switch to Repostory Database Authentication. 
1. Run the cell below. 
2. Click the link in the cell.

In [23]:
print(database+'/console/?mode=compact#settings/authentication')

http://services-uscentral.skytap.com:18638/console/?mode=compact#settings/authentication


To support multiple users you need a way to authenticate those other users. There are two choices. 
1. **Repository Database Authentication**. This delegates the authentication of console users to the Db2 repository database used by the console to store historical monitoring data. 
2. **LDAP**. This delegates the authentication to an external LDAP service.
In this example we use the Repository Database to authenticate console users. You associate console users and administrators to Db2 Authorities, Groups, UDFs or Role. 

In this lab we are going to setup the first option, Repository Database Authentication. 

Before switching to Repository Database Authentication lets create some new users in the operating system that Db2 can recognize. 

### Adding Operating System Users
The repository database uses standard Db2 authentiation. The simplest way to set this up is to use the userid authentication that is built into the operating system that your Db2 database is running on. 

To add new users to the console you need to add new users to your operating system. 

In this hands-on lab you can create those new users through the Linux settings console. 

1. Click the **Settings** Icon at the bottom left of the screen. 
1. Click **Details** in the meu on the left side of the page
1. Click **Users** in the menu on the left side of the page
1. Click **Unlock** at the top right side of the page
1. Enter the DB2POT password **123qwe123**.
1. Click **Add User...** at the top-right of the page
1. Type **Peter** into the **Full Name** and the **Username** fields.
1. Click **Set a password now**
1. Enter **DataConsole** into the **password fields**
1. Click **Add** at the top-right of the page

Repeat steps 5 to 9 for the following new users:
1. **Paul**
1. **Mary**

### Creating Roles for Authentication
Now that you have three new users we need to create two database roles to classify the access allowd for each id. Either an Admin or User role (CONSOLE_ADM, CONSOLE_USR). 

In [10]:
user = 'DB2INST1'
password = 'db2inst1'
profile = 'Repository'
    
script = 'CREATE ROLE CONSOLE_ADM; CREATE ROLE CONSOLE_USR'        
print('Adding Roles')
displayResults(runSQL(profile, user, password, script))
print('done')

Adding Roles


KeyError: 'id'

### Adding Users to the Console using Repository Authentication
Now that you have three new users lets add them grant them database authorities and CONSOLE roles. The following cell will grant a user to either an Admin or User role (CONSOLE_ADM, CONSOLE_USR). 

In [ ]:
userList = {'newUser':['PETER', 'PAUL', 'MARY'], 'Type':['Admin', 'User', 'User']} 
userListDF = pd.DataFrame(userList) 
display(userListDF)
  
user = 'DB2INST1'
password = 'db2inst1'
profile = 'HISTORY'

for row in range(0, len(userListDF)):
    Type = userListDF['Type'].iloc[row]
    newUser = userListDF['newUser'].iloc[row]
    
    script = 'GRANT DBADM, CREATETAB, BINDADD, CONNECT, CREATE_NOT_FENCED, IMPLICIT_SCHEMA, LOAD ON DATABASE TO USER "'+newUser+'";'
    if userListDF['Type'].iloc[row] == 'Admin' :
        script = script + 'GRANT ROLE "CONSOLE_ADM" TO USER "'+newUser+'" WITH ADMIN OPTION;'
    else : 
        script = script + 'GRANT ROLE "CONSOLE_USR" TO USER "'+newUser+'";'        
    print('Adding User: '+newUser)
    displayResults(runSQL(profile, user, password, script))
print('done')

### Switch Authentication Method in the Console
Next that we can users that are assigned to the correct role, we can switch from using the single default user to using Repository Database authentication for multiple users. You can do this through the Db2 Data Management Console and navigating to the **Repository Setting** page or by running the next cell below. Pick one of the two methods below to get to the Repository Settings page. Then follow the 10 steps to change the authentication method.

Use the full console to switch to Repository Database Authentication:
1. Click the link http://localhost:11080/console
2. Click the gear icon at the top right of the page
3. Select **Authentication**

Use the embedded page from the console to switch to Repostory Database Authentication
1. Run the cell below

Now configure the console to use the Repository database to authenticate users. 
1. Select **Repository** from the list of available Authentiation Types.
2. Scroll down and select **Db2 Roles** from the list of User role matting methods
3. Enter CONSOLE_ADM in the Admin roles field
4. Enter CONSOLE_USR in the User roles field
5. Click Next
6. Enter **Peter** in the Test user ID field
7. Enter **DataConsole** in the Test user password field
8. Click **Test**. You should see confirmation that the test succeeded
9. Select **Save** at the bottom right of the page. A confirmation dialog appears.
10. Click **Yes**. You should now see that Repository authentication is set and enabled successfully


In [ ]:
URL = database+'/console/?mode=compact#settings/authentication'
print(URL)
IFrame(URL, width=1400, height=300)

### Test a New Console User
To test the new user and authentication method you can log out and into the console and you can run the cell below. After signing in you run SQL for the first time. This prompts a new user to enter a userid and password to authenticate against the Db2 database. You can save this authentication pair if you want to run SQL through the service later.

1. Open the console http://localhost:11080/console
2. Click the user icon at the very top right of the screen
3. Select **sign out** a confirmation dialog appears
4. Select **Yes**
5. Enter **Peter** in the user field
6. Enter **DataConsole** in the password field
7. Click **History** Connection name in the Databases homepage
8. Click the **Run SQL** icon at the left of the page
9. Enter **select TABSCHEMA, TABNAME from syscat.tables**
9. Click **Run All**
9. Enter **Peter** and **DataConsole** in the userid and password field
10. IMPORTANT - Click **Save Credentials to repository** 
11. Click **Test Connection**
12. Click **Save**

This next cell runs a simple query through the console service using one of the new users. This only works if you chose to save Peter's credentials to the repository.

In [ ]:
sql = 'select TABSCHEMA, TABNAME from syscat.tables'
displayResults(runSQL('HISTORY', 'Peter', 'DataConsole', sql))

### Grant Authority to DBINST1 to Access the Console
When you added **Peter** to the CONSOLE_ADM role you also included the **WITH ADMIN OPTION** that means that **Peter** can now grant other users access to the **CONSOLE_ADM** group. Run the SQL in the next cell to grant access to DB2INST to the Console as an Administrator.

In [ ]:
sql = 'GRANT ROLE "CONSOLE_ADM" TO USER "DB2INST1"'
displayResults(runSQL('HISTORY', 'Peter', 'DataConsole', sql))

Now lets try running the previous select statement using the **DB2INST1** user id.

In [ ]:
sql = 'select TABSCHEMA, TABNAME from syscat.tables'
displayResults(runSQL('HISTORY', 'db2inst1', 'db2inst1', sql))

## Manage Console User Priviledges
In the console you can control which users can access specific database connections. These are divided into Owners, who can grant privleges, and users who can only use the connection.

### Get User Connection Profile Privileges
The next cell lets you control exactly which users can access specific databases. You can also access this through the console **Users and Privileges** settings page in the main console. 

In [ ]:
URL = database+'/console/?mode=compact#settings/users'
print(URL)
IFrame(URL, width=1400, height=300)

If you have a lot to users and databases to manage it may be useful to script these changes. The next cell includes an example of how to retrieve the current privileges through the console APIs. 

In [ ]:
# Get User Connection Profile Privleges
r = databaseAPI.getUserPriviledges()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)))
else:
    print(databaseAPI.getStatusCode(r))      

### Change User Connection Profile Privileges
If you have a lot of changes to make at once, the following cells are examples of how to script changes to multiple user and database connection priviledges. The first two cells in this section define reusable routines that let you add and revoke privileges with a single call. 

In [ ]:
# Change User Connection Profile Privileges
def addProfilePrivileges(profile, name, userType) :

    if userType == 'user' : 
        r = databaseAPI.assignUserPrivileges(profile, name)
    else: 
        r = databaseAPI.assignOwnerPrivileges(profile, name)

    if (databaseAPI.getStatusCode(r)==201):
        print(name+' added to: '+profile+" as a new "+userType+".")
    else:
        print(databaseAPI.getStatusCode(r)) 
    
print('Created routine to add profile privileges')

In [ ]:
# Revoke User Connection Profile Privileges
def revokeProfilePrivileges(profile, name) :

    r = databaseAPI.revokeProfilePrivileges(profile, name)

    if (databaseAPI.getStatusCode(r)==201):
        print(name+' privilege revoked from: '+profile)
    else:
        print(databaseAPI.getStatusCode(r)) 
    
print('Created routine to revoke profile privileges')

In [ ]:
addProfilePrivileges('HISTORY', 'PAUL', 'owner')

In [ ]:
revokeProfilePrivileges('HISTORY', 'PAUL')

These next two examples, show how to process an entire list of changes through a simple loop. Both to grant and revoke user privileges. 

In [ ]:
userList = {'userName':['PETER', 'PAUL', 'MARY', 'PETER', 'PAUL', 'MARY'], 'Profile':['HISTORY', 'HISTORY', 'HISTORY', 'SAMPLE', 'SAMPLE','SAMPLE'],'Type':['owner', 'user', 'user', 'owner', 'user', 'user']} 
userListDF = pd.DataFrame(userList) 
display(userListDF)

for row in range(0, len(userListDF)):
    profile = userListDF['Profile'].iloc[row]
    userType = userListDF['Type'].iloc[row]
    name = userListDF['userName'].iloc[row]   
    addProfilePrivileges(profile, name, userType)
print('done')

In [ ]:
userList = {'userName':['PETER', 'PAUL', 'MARY', 'PETER', 'PAUL', 'MARY'], 'Profile':['HISTORY', 'HISTORY', 'HISTORY', 'SAMPLE', 'SAMPLE','SAMPLE']} 
userListDF = pd.DataFrame(userList) 
display(userListDF)

for row in range(0, len(userListDF)):
    profile = userListDF['Profile'].iloc[row]
    name = userListDF['userName'].iloc[row]   
    revokeProfilePrivileges(profile, name)
print('done')


## Managing Database Connection Profiles
To monitor and manage more Db2 databases through the console, you need to add a new connection profile for each database. The connection profile includes information on how to connect to the database (host, port, and database name) as well as log in information for monitoring and management. 

### Adding Connection Profiles

Let's start by checking the list of current database profiles available in the console. 

Run the next two cells. 

The first cell opens the Connection Profile setting page. You can access this through the full console (http://localhost:11080/console) by selecting **Connection Profile** from the settings menu at the top right of the screen. 

The second cell lists the connection information by accessing the console API.

In [ ]:
URL = database+'/console/?mode=compact#connection/manager'
print(URL)
IFrame(URL, width=1400, height=300)

In [ ]:
# Get Console Connection Profiles
r = databaseAPI.getConnectionProfiles()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)))
else:
    print(databaseAPI.getStatusCode(r))      

### Export Connection Profile List
To copy information from one instance of the Db2 Data Management Console to another you can export the current connection profile by selecting the **up arrow** icon at the top right of the Connection Profile page. You can then import the same file back into another console to move connection information from on console to another. 

You can also export the connection profile list through the console API. The example cells below create a csv file with the full connection profile information. You can then read back the csv file to see the contents of the csv file. 

In [ ]:
# Export Console Connection Profiles
r = databaseAPI.getExportConnectionProfiles()

if (databaseAPI.getStatusCode(r)==200):
    with open('connectionProfile.csv','wb') as file:
        file.write(r.content)
        print('Created connectionProfile.csv')
else:
    print(databaseAPI.getStatusCode(r))  
    

In [ ]:
# Import Console Connection Profile
df = pd.read_csv("connectionProfile.csv") 
df = df.drop(df.index[1]) #Drop the second row
df = df.iloc[:, :-1] # Drop the last column
display(df)

Scroll to the right and find the he **collectionCred** column.

This is an encrypted version of the userid and password used to connect to the database.
Anytime that the Db2 Data Management Console stores a userid and password, it encrypts it using a routine included with the console. If you export and import a connection profile you can choose to export with our without credentials. If you choose to include the credentials you can use the same userid and passwords from the original connection profile list when you import into a new console. 

### Create a new Db2 Database
In this next section you will create a new and add it to the Db2 Data Management Console. 

To create a new Db2 database you need to access the Db2 Terminal:

1. Click the **Terminal** icon at the bottom left of the screen
2. Switch to the DB2INST1 USER by running the following command
```
su - db2inst1
```
3. Enter the password **db2inst1**
4. Create a database with the following command where **dbone** is the name of the new database
```
db2 create database dbone
```

### Create a new Database Connection Profile through the User Interface
You can add the database through the Connection Profile page or though the console API. Let's try with the console first. Either open the Connection Profile page through the full console (http://localhost:11080/console) and select **Connection Profile** from the settings menu at the top right of the screen, or run the cell below. 

In [ ]:
IFrame(database+'/console/?mode=compact#connection/manager/add', width=1400, height=700)

To create a new connection:
1. Enter **databaseOne** in the **Connection name** field
2. Enter **localhost** in the **Host** field
3. Enter **50000** in the **Port** field
4. Enter **dbone** in the **Database** field
5. Enter **db2inst1** in the **User name** field
6. Enter **123qwe123** in the **Password** field
7. Select **Enable operation**
8. Enter **db2inst1** in the **User name** field
9. Enter **123qwe123** in the **Password** field
10. Select **Save credentials to repository**
11. Click **Save**

Now, lets delete the connection you just created so we can recreate it using the console API.
1. Click the checkbox beside **databaseone**
2. Click **delete**
3. Click **Yes** to confirm that you want to delete the databaseone connection. This does not drop the actual database. 

### List the current Connection Profiles
Now we can perform the same actions using the console API. Let's start by listing the current database connections. Run the next cell.

In [ ]:
# Get Console Connection Profiles
r = databaseAPI.getConnectionProfiles()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)))
else:
    print(databaseAPI.getStatusCode(r))      

### Check Connection Information without Creating a new Profile
Just like the user interface has a **Test connection** button you can test that your userid and password connects to the database before you create a connection profile. The next cell will check that the following connection information is valid. 

In [ ]:
dbName = 'dbone'
port = '50000'
host = 'localhost'
userid = 'db2inst1'
password = 'db2inst1'
r = databaseAPI.postTestConnection(dbName, port, host, userid, password)
if (databaseAPI.getStatusCode(r)==200):
    print(databaseAPI.getJSON(r))
else:
    print(databaseAPI.getStatusCode(r)) 

### Create a new Database Connection Profile
The next cell adds a connection to the **dbone** database. 

In [ ]:
connectionName = 'databaseOne'
dbName = 'dbone'
port = '50000'
host = 'localhost'
userid = 'db2inst1'
password = 'db2inst1'
comment = 'New connection profile test'
r = databaseAPI.postConnectionProfile(connectionName, dbName, port, host, userid, password, comment)
if (databaseAPI.getStatusCode(r)==201):
    print("Created connection profile "+connectionName)
else:
    print(databaseAPI.getStatusCode(r)) 

### List the current Connection Profiles
Next run the routine to list the connection profiles to confirm that the connection to dbone is now available. 

In [ ]:
# Get Console Connection Profiles
r = databaseAPI.getConnectionProfiles()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)))
else:
    print(databaseAPI.getStatusCode(r))      

### Update an Existing Database Connection Profile
You can also update an exsiting connection profile. The next cell is an example of updating the comment field. You can use the same routine to update the userid and password, host or portnumber as well.

In [ ]:
connectionName = 'databaseOne'
dbName = 'dbone'
port = '50000'
host = 'localhost'
userid = 'db2inst1'
password = 'db2inst1'
comment = 'Updated Comment'
r = databaseAPI.putConnectionProfileUpdate(connectionName, dbName, port, host, userid, password, comment)
if (databaseAPI.getStatusCode(r)==200):
    print("Updated connection profile "+connectionName)
else:
    print(databaseAPI.getStatusCode(r)) 

Check that the comment field has been updated with the new comment. Run the cell below and check the comment field near the bottom of the list. 

In [ ]:
# Check Connection Profile
connectionName = 'databaseOne'

r = databaseAPI.getConnectionProfile(connectionName)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)).transpose())
else:
    print(databaseAPI.getStatusCode(r))      

### Delete a connection profile
Finally you can delete existing database connection profiles from the console using a single API call. 

In [ ]:
connectionName = 'databaseOne'

r = databaseAPI.deleteConnectionProfile(connectionName)
if (databaseAPI.getStatusCode(r)==200):
    print("Deleted connection profile "+connectionName)
else:
    print(databaseAPI.getStatusCode(r)) 

## Manage Monitoring Profiles
Each monitored database is associated with a monitoring profile. The profile defines what kind of data is collected, when it is collected and how long it is retained in the repository database. The monitoring profile also configures which alerts are enabled.

Multiple database can be associated with each profile. There is also one default profile that any new database connection profile is added to when it is created. 

### List Monitor Profiles
Let's start by listing the current monitoring profiles. 

You can access this through the full console (http://localhost:11080/console) by selecting **Monitoring Profile** from the settings menu at the top right of the screen. You can also go directly to the monitoring profiles page by running the next cell. 

1. Click the elipses (...) in the **Actions** column at the right side of the list and click **Edit**
2. Browse through the monitoring profile page to see the default settings for this database.
3. Click **Cancel** to return to the monitoring profile list.

In [ ]:
URL = database+'/console/?mode=compact#monitorprofile'
print(URL)
IFrame(URL, width=1400, height=300)

### Creating a New Monitoring Profile
Use the user interface to create a new profile. 
1. Click **New profile** at the top right of the screen
2. Enter **Test Profile** in the **Profile name (New)** field
3. Click **Select database list**
4. Click the checkbox beside **SAMPLE**
5. Click **Save** to add asign the SAMPLE database to the new monitoring profile
6. Click **Save** to create the new profile

### Using the API to access Monitoring Profiles
You can access the same information throug the console API. The next cell lists all the available monitoring profiles. 

In [ ]:
r = databaseAPI.getMonitoringProfiles()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r) 
    profileList = pd.DataFrame(json_normalize(json['resources']))
    display(profileList)
else:
    print(databaseAPI.getStatusCode(r)) 

### Get a Monitor Profile
To get the details of a single monitoring profile, run the cell below. The complete JSON string that is returned can be long. So we only display some key information. However you can uncomment the line that prints the entire JSON string if you want to see everything.

In [ ]:
profileName = "Test Profile"

# Find the identifier used to access this profile
profileID = str(databaseAPI.getProfileIndex(profileName))

# Retrieve the monitoring profile information for the identifier
r = databaseAPI.getMonitoringProfile(profileID)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r) 
    # print(json)
    # Print out the base information of the monitoring profile
    print(json['base_info'])
    # Print the common monitoring settings
    print(json['monitor_config']['common_settings'])
else:
    print(databaseAPI.getStatusCode(r))    

### Change a Monitoring Profile
Now that we have the complete monitoring profile as a JSON string, we can update the string and put it back to update the monitoring profile settings. In this example you change the database included in the profile and change the data collection period. 

In [ ]:
profileName = "Test"

# Get the existing json for the monitoring profile
profileID = str(databaseAPI.getProfileIndex(profileName))
r = databaseAPI.getMonitoringProfile(profileID)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r) 
else:
    print(databaseAPI.getStatusCode(r))    

# Change the json that describes the list of databases assigned to this monitoring profile
json['base_info']['assign_database_list'] = "HISTORY"
json['monitor_config']['common_settings']['collect_data_every'] = 10  

# Put the updated profile back
r = databaseAPI.putMonitoringProfile(profileID, json)
if (databaseAPI.getStatusCode(r)==200):
    print(profileName+' Monitoring Profile Updated')
    print(json['base_info'])
    print(json['monitor_config']['common_settings'])
else:
    print(databaseAPI.getStatusCode(r)) 

## Cleaning up

### Re-enable the original setup administrator account

The setup administrator account is automatically disabled when the administrator account is configured the authentication settings. However, during emergency situations (for example, when the configured LDAP server is down, or the entries of administrator accounts are removed or renamed) when none of the administrator account is available to log into the console, it becomes necessary to re-enable the setup administrator account.

To re-enable setup administrator account:
1. Click the **Files** icon at the bottom left of the screen
2. Select the **Home** directory
3. Select the **dmc** directory
3. Select the **Config** directory
4. Select the **dewebserver_override.properties** file
5. Hit the **Enter** key
6. In the file editor change the property value named **auth** to **superAdmin** (auth=superAdmin, no spaces before or after)
7. Click **Save** at the top right of the editor screen
8. If necessary log out of the IBM Db2 Data Management Console and log in with the setup administrator account.
    Userid: db2inst1
    Password: db2inst1
9. Reload the Db2 Console browser page
10. Navigate to Authentication Setting page and configure the authentication settings. You should see that the settings have returned to the default configuration.

#### Credits: IBM 2019-2021, Peter Kohlmann [kohlmann@ca.ibm.com]